In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import requests
import pymongo
from datetime import datetime
import time

In [2]:
file = "data/Denver/crime.csv"
data1 = pd.read_csv(file)

In [3]:
# INCIDENT_ID	OFFENSE_ID	OFFENSE_CODE	OFFENSE_CODE_EXTENSION	OFFENSE_TYPE_ID	OFFENSE_CATEGORY_ID	FIRST_OCCURRENCE_DATE	
# LAST_OCCURRENCE_DATE	REPORTED_DATE	INCIDENT_ADDRESS	GEO_X	GEO_Y	GEO_LON	GEO_LAT	DISTRICT_ID	PRECINCT_ID	NEIGHBORHOOD_ID	IS_CRIME	IS_TRAFFIC
data = data1[["INCIDENT_ID", "FIRST_OCCURRENCE_DATE", "OFFENSE_TYPE_ID", "OFFENSE_CATEGORY_ID", "GEO_LAT", "GEO_LON"]].copy()
data.rename(columns={'GEO_LAT': 'latitude', 'GEO_LON': 'longitude'}, inplace=True)
data

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude
0,2016376978,6/15/2016 11:31:00 PM,weapon-unlawful-discharge-of,all-other-crimes,39.773188,-104.809881
1,20186000994,10/11/2017 12:30:00 PM,theft-other,larceny,39.785649,-104.781434
2,20166003953,3/4/2016 8:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.663490,-104.957381
3,201872333,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440
4,2017411405,6/22/2017 8:53:00 PM,theft-shoplift,larceny,39.717107,-104.955370
5,201872837,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928
6,20186001015,6/1/2017 12:15:00 PM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.732790,-105.025543
7,201870628,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794
8,201870833,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650
9,20186001048,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451


In [4]:
dates = []
years = []

for date in data["FIRST_OCCURRENCE_DATE"]:
    date = datetime.strptime(date, "%m/%d/%Y %I:%M:%S %p")
    dates.append(date)
    years.append(date.year)
    
data["date"] = dates
data["years"] = years
data

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,date,years
0,2016376978,6/15/2016 11:31:00 PM,weapon-unlawful-discharge-of,all-other-crimes,39.773188,-104.809881,2016-06-15 23:31:00,2016
1,20186000994,10/11/2017 12:30:00 PM,theft-other,larceny,39.785649,-104.781434,2017-10-11 12:30:00,2017
2,20166003953,3/4/2016 8:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.663490,-104.957381,2016-03-04 20:00:00,2016
3,201872333,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440,2018-01-30 19:20:00,2018
4,2017411405,6/22/2017 8:53:00 PM,theft-shoplift,larceny,39.717107,-104.955370,2017-06-22 20:53:00,2017
5,201872837,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928,2018-01-31 00:44:00,2018
6,20186001015,6/1/2017 12:15:00 PM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.732790,-105.025543,2017-06-01 12:15:00,2017
7,201870628,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794,2018-01-30 07:40:00,2018
8,201870833,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650,2018-01-30 09:10:00,2018
9,20186001048,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451,2018-01-31 00:55:00,2018


In [ ]:
print(type(data["years"][0]))
data

In [5]:
data_2014 = pd.DataFrame()
data_2015 = pd.DataFrame()
data_2016 = pd.DataFrame()
data_2017 = pd.DataFrame()
data_2018 = pd.DataFrame()

data_2014 = data.loc[data["years"] == 2014]
print(data_2014)
data_2015 = data.loc[data["years"] == 2015]
print(data_2015)
data_2016 = data.loc[data["years"] == 2016]
print(data_2016)
data_2017 = data.loc[data["years"] == 2017]
print(data_2017)
data_2018 = data.loc[data["years"] == 2018]
data_2018

        INCIDENT_ID   FIRST_OCCURRENCE_DATE       OFFENSE_TYPE_ID  \
408       201871332  12/19/2014 12:00:00 PM        theft-embezzle   
11512    2018137970   12/1/2014 12:00:00 AM     sex-aslt-non-rape   
12722    2018107649    2/13/2014 9:24:00 PM  criminal-trespassing   
18979    2014478770   9/10/2014 11:08:00 AM      traffic-accident   
21913    2018345525     5/1/2014 1:00:00 PM        assault-simple   
...             ...                     ...                   ...   
474713  20176011247  11/19/2014 12:25:00 PM  fraud-identity-theft   
483431   2018553995     8/7/2014 8:45:00 PM     threats-to-injure   
487237   2018776994    4/17/2014 6:00:00 PM         sex-aslt-rape   
493148   2018718756    9/24/2014 1:23:00 PM        forgery-checks   
502702   2018669371    5/1/2014 12:00:00 AM     sex-aslt-non-rape   

                 OFFENSE_CATEGORY_ID   latitude   longitude  \
408               white-collar-crime  39.823944 -104.770129   
11512                 sexual-assault        N

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,date,years
3,201872333,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440,2018-01-30 19:20:00,2018
5,201872837,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928,2018-01-31 00:44:00,2018
7,201870628,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794,2018-01-30 07:40:00,2018
8,201870833,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650,2018-01-30 09:10:00,2018
9,20186001048,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451,2018-01-31 00:55:00,2018
10,201872390,1/30/2018 8:04:00 PM,drug-methampetamine-possess,drug-alcohol,39.678463,-104.925197,2018-01-30 20:04:00,2018
11,20186001033,1/28/2018 5:25:00 AM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.694578,-104.997720,2018-01-28 05:25:00,2018
17,201869174,1/29/2018 2:50:00 PM,criminal-trespassing,all-other-crimes,39.740117,-104.998855,2018-01-29 14:50:00,2018
18,201869047,1/4/2018 12:00:00 PM,fraud-by-use-of-computer,white-collar-crime,39.701835,-104.980747,2018-01-04 12:00:00,2018
19,201871381,1/30/2018 12:38:00 PM,traffic-accident,traffic-accident,39.775833,-104.865745,2018-01-30 12:38:00,2018


In [6]:
# def reformat_datetime(df):
#     datetimes = []
#     for d,t  in zip(df["date"], df["Time Occurred"]):
# #         print(d)
# #         print(type(d))
#     #     datetime = (datetime.strftime(d, format='%Y-%m-%d') + " " + str(t))
#         d = datetime.strptime(str(d), "%Y-%m-%d %H:%M:%S")
#         date_time = datetime.strftime(d, format='%Y-%m-%d') + "T" + str((int(t/100))) + ":00:00"
#     #     datetime = d + " " + t
#         datetimes.append(date_time)
#     df["date_time"] = datetimes
#     return df

def reformat_datetime(df):
    datetimes = []
    for d in df["date"]:
        date_time = datetime.strftime(d, format='%Y-%m-%dT%H:00:00')
        datetimes.append(date_time)
    df["date_time"] = datetimes
    return df

In [7]:
df_years = [data_2014, data_2015, data_2016, data_2017, data_2018]

for i, yr in enumerate(df_years):
    df = reformat_datetime(yr)
    if i == 0:
        data_2014_1 = df.copy()
    elif i ==1:
        data_2015_1 = df.copy()
    elif i ==2:
        data_2016_1 = df.copy()
    elif i ==3:
        data_2017_1 = df.copy()
    elif i ==4:
        data_2018_1 = df.copy()


/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy





----

# Final DFs per year

In [8]:
data_2014_1

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,date,years,date_time
408,201871332,12/19/2014 12:00:00 PM,theft-embezzle,white-collar-crime,39.823944,-104.770129,2014-12-19 12:00:00,2014,2014-12-19T12:00:00
11512,2018137970,12/1/2014 12:00:00 AM,sex-aslt-non-rape,sexual-assault,NaN,NaN,2014-12-01 00:00:00,2014,2014-12-01T00:00:00
12722,2018107649,2/13/2014 9:24:00 PM,criminal-trespassing,all-other-crimes,39.694575,-104.989849,2014-02-13 21:24:00,2014,2014-02-13T21:00:00
18979,2014478770,9/10/2014 11:08:00 AM,traffic-accident,traffic-accident,39.674578,-104.973130,2014-09-10 11:08:00,2014,2014-09-10T11:00:00
21913,2018345525,5/1/2014 1:00:00 PM,assault-simple,other-crimes-against-persons,39.681391,-104.958938,2014-05-01 13:00:00,2014,2014-05-01T13:00:00
27164,20186003087,7/1/2014 12:30:00 AM,fraud-identity-theft,white-collar-crime,39.769633,-105.031377,2014-07-01 00:30:00,2014,2014-07-01T00:00:00
31347,2019115914,2/21/2014 3:00:00 PM,theft-stln-vehicle-trailer,auto-theft,39.782826,-104.909287,2014-02-21 15:00:00,2014,2014-02-21T15:00:00
34155,2018860988,12/8/2014 1:27:00 AM,weapon-fire-into-occ-bldg,aggravated-assault,39.682613,-105.017685,2014-12-08 01:27:00,2014,2014-12-08T01:00:00
39610,20195001857,5/11/2014 12:00:00 AM,sex-aslt-rape,sexual-assault,NaN,NaN,2014-05-11 00:00:00,2014,2014-05-11T00:00:00
51502,2014125067,3/11/2014 8:00:00 AM,aggravated-assault,aggravated-assault,39.755872,-105.025426,2014-03-11 08:00:00,2014,2014-03-11T08:00:00


In [9]:
data_2015_1

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,date,years,date_time
730,20186001823,2/19/2015 8:50:00 AM,fraud-identity-theft,white-collar-crime,39.750645,-104.981133,2015-02-19 08:50:00,2015,2015-02-19T08:00:00
1153,2016504455,8/9/2015 8:00:00 AM,theft-of-services,larceny,39.673662,-104.969627,2015-08-09 08:00:00,2015,2015-08-09T08:00:00
1905,2015116121,3/1/2015 8:30:00 AM,homicide-other,murder,39.650783,-104.904484,2015-03-01 08:30:00,2015,2015-03-01T08:00:00
10863,2018427131,1/1/2015 12:00:00 AM,sex-aslt-rape,sexual-assault,NaN,NaN,2015-01-01 00:00:00,2015,2015-01-01T00:00:00
12645,2016587443,10/15/2015 3:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.762523,-104.919806,2015-10-15 15:00:00,2015,2015-10-15T15:00:00
12658,20188004793,10/13/2015 6:00:00 AM,theft-other,larceny,39.849292,-104.673812,2015-10-13 06:00:00,2015,2015-10-13T06:00:00
13943,2018100576,6/1/2015 12:00:00 PM,sex-aslt-non-rape-pot,sexual-assault,NaN,NaN,2015-06-01 12:00:00,2015,2015-06-01T12:00:00
15095,2018189375,9/1/2015 9:00:00 PM,sex-aslt-rape,sexual-assault,NaN,NaN,2015-09-01 21:00:00,2015,2015-09-01T21:00:00
16573,201866698,10/26/2015 12:00:00 PM,forgery-other,white-collar-crime,39.777433,-104.857014,2015-10-26 12:00:00,2015,2015-10-26T12:00:00
17160,201639045,12/19/2015 3:00:00 PM,criminal-mischief-mtr-veh,public-disorder,39.702770,-104.913305,2015-12-19 15:00:00,2015,2015-12-19T15:00:00


In [10]:
data_2016_1

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,date,years,date_time
0,2016376978,6/15/2016 11:31:00 PM,weapon-unlawful-discharge-of,all-other-crimes,39.773188,-104.809881,2016-06-15 23:31:00,2016,2016-06-15T23:00:00
2,20166003953,3/4/2016 8:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.663490,-104.957381,2016-03-04 20:00:00,2016,2016-03-04T20:00:00
12,2016340553,5/31/2016 4:54:00 PM,theft-shoplift,larceny,39.689021,-104.942153,2016-05-31 16:54:00,2016,2016-05-31T16:00:00
14,2016236602,4/16/2016 3:25:00 PM,theft-shoplift,larceny,39.699214,-104.902812,2016-04-16 15:25:00,2016,2016-04-16T15:00:00
29,2016398281,6/12/2016 8:00:00 PM,criminal-trespassing,all-other-crimes,39.789301,-104.883245,2016-06-12 20:00:00,2016,2016-06-12T20:00:00
31,2016309815,5/18/2016 12:15:00 PM,theft-of-motor-vehicle,auto-theft,39.723824,-104.993159,2016-05-18 12:15:00,2016,2016-05-18T12:00:00
36,2016256044,4/24/2016 10:00:00 PM,theft-of-motor-vehicle,auto-theft,39.673603,-104.986690,2016-04-24 22:00:00,2016,2016-04-24T22:00:00
44,2016229783,4/13/2016 1:00:00 PM,theft-other,larceny,39.707683,-104.874852,2016-04-13 13:00:00,2016,2016-04-13T13:00:00
45,2016330967,5/27/2016 12:00:00 PM,theft-from-bldg,larceny,39.717317,-104.954910,2016-05-27 12:00:00,2016,2016-05-27T12:00:00
50,20166003736,2/28/2016 6:30:00 AM,criminal-mischief-other,public-disorder,39.719075,-104.943407,2016-02-28 06:30:00,2016,2016-02-28T06:00:00


In [11]:
data_2017_1

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,date,years,date_time
1,20186000994,10/11/2017 12:30:00 PM,theft-other,larceny,39.785649,-104.781434,2017-10-11 12:30:00,2017,2017-10-11T12:00:00
4,2017411405,6/22/2017 8:53:00 PM,theft-shoplift,larceny,39.717107,-104.955370,2017-06-22 20:53:00,2017,2017-06-22T20:00:00
6,20186001015,6/1/2017 12:15:00 PM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.732790,-105.025543,2017-06-01 12:15:00,2017,2017-06-01T12:00:00
13,2017383968,2/24/2017 4:55:00 PM,theft-of-motor-vehicle,auto-theft,39.754477,-105.023330,2017-02-24 16:55:00,2017,2017-02-24T16:00:00
15,20176004695,5/28/2017 2:00:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.764565,-105.014275,2017-05-28 02:00:00,2017,2017-05-28T02:00:00
16,2017395809,6/16/2017 9:30:00 PM,theft-of-motor-vehicle,auto-theft,39.675105,-104.910573,2017-06-16 21:30:00,2017,2017-06-16T21:00:00
25,201871768,10/30/2017 12:00:00 PM,theft-embezzle,white-collar-crime,39.652260,-104.901787,2017-10-30 12:00:00,2017,2017-10-30T12:00:00
28,2017294559,5/6/2017 12:00:00 AM,theft-of-motor-vehicle,auto-theft,39.737471,-105.032978,2017-05-06 00:00:00,2017,2017-05-06T00:00:00
35,2017510328,8/1/2017 8:34:00 PM,aggravated-assault,aggravated-assault,39.741821,-104.987834,2017-08-01 20:34:00,2017,2017-08-01T20:00:00
47,2017791814,11/25/2017 6:30:00 PM,assault-simple,other-crimes-against-persons,39.746273,-104.964369,2017-11-25 18:30:00,2017,2017-11-25T18:00:00


In [12]:
data_2018_1

,INCIDENT_ID,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,date,years,date_time
3,201872333,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440,2018-01-30 19:20:00,2018,2018-01-30T19:00:00
5,201872837,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928,2018-01-31 00:44:00,2018,2018-01-31T00:00:00
7,201870628,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794,2018-01-30 07:40:00,2018,2018-01-30T07:00:00
8,201870833,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650,2018-01-30 09:10:00,2018,2018-01-30T09:00:00
9,20186001048,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451,2018-01-31 00:55:00,2018,2018-01-31T00:00:00
10,201872390,1/30/2018 8:04:00 PM,drug-methampetamine-possess,drug-alcohol,39.678463,-104.925197,2018-01-30 20:04:00,2018,2018-01-30T20:00:00
11,20186001033,1/28/2018 5:25:00 AM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.694578,-104.997720,2018-01-28 05:25:00,2018,2018-01-28T05:00:00
17,201869174,1/29/2018 2:50:00 PM,criminal-trespassing,all-other-crimes,39.740117,-104.998855,2018-01-29 14:50:00,2018,2018-01-29T14:00:00
18,201869047,1/4/2018 12:00:00 PM,fraud-by-use-of-computer,white-collar-crime,39.701835,-104.980747,2018-01-04 12:00:00,2018,2018-01-04T12:00:00
19,201871381,1/30/2018 12:38:00 PM,traffic-accident,traffic-accident,39.775833,-104.865745,2018-01-30 12:38:00,2018,2018-01-30T12:00:00


In [13]:
# .to_csv(index=False)
data_2014_1.to_csv (r'data/Denver/data_2014.csv', index = None, header=True)
data_2015_1.to_csv (r'data/Denver/data_2015.csv', index = None, header=True)
data_2016_1.to_csv (r'data/Denver/data_2016.csv', index = None, header=True)
data_2017_1.to_csv (r'data/Denver/data_2017.csv', index = None, header=True)
data_2018_1.to_csv (r'data/Denver/data_2018.csv', index = None, header=True)